In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# we can access the API token from .env file
HF_API_TOKEN = os.getenv("HF_API_TOKEN")

In [ ]:
# We'll use HuggingFace's embeddings which are free and open source

In [22]:
%pip install langchain chromadb sentence-transformers pypdf langchain-community

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   --------- ------------------------------ 2.6/11.6 MB 13.7 MB/s eta 0:00:01
   ---------------------- ----------------- 6.6/11.6 MB 16.8 MB/s eta 0:00:01
   ------------------------------------- -- 11.0/11.6 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 18.2 MB/s  0:00:00
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
    --------------------------------------- 4.2/241.3 MB 22.9 MB/s eta 0:00:11
   - -------------------------------------- 7.9/241.3 MB 19.4 MB/s eta 0:00:13
   - -------------------------------------- 10.5/241.3 MB 17.2 MB/s eta 0:00:14
   -- ------------------------------------- 15.5/241.3 MB 19.1 MB/s eta 0:00:12
   --- ------------------------------------ 21.2/241.3 MB 21.0 MB/s eta 0:00:11
   ---- ----------------------------------- 27.0/241.3 MB 22.5 MB/s eta 0:00:10
   ----- ---------------------------------- 33.0/241.3 MB 23.3 MB/s et

In [23]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [24]:
from langchain.schema import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [35]:
# Create list of documents
docs = [doc1, doc2, doc3, doc4, doc5]

# Initialize HuggingFace embeddings (open source)
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Create a new Chroma vector store
vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./chroma_db"  # This will store the database locally
)

In [26]:
# If you need to add more documents later, you can use add_documents
new_doc = Document(
    page_content="Andre Russell is a powerful all-rounder known for his explosive batting and crucial bowling spells. Playing for Kolkata Knight Riders, he's won many matches single-handedly.",
    metadata={"team": "Kolkata Knight Riders"}
)
vector_store.add_documents([new_doc])

['6f2fecdc-2444-45d4-9569-20394f20279d']

In [27]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['943b6c66-f7b9-4633-acc6-abba5a79cd69',
  'aa1921e6-a466-48e3-b6dc-f32ccfffb8de',
  '1239c316-d885-4baf-b588-f1df1c7b50d7',
  '4f3b4fd0-7036-46f1-b5e7-35327fbf69b7',
  '7e2401b2-7054-436d-b6e1-2f164217c5e1',
  '6f2fecdc-2444-45d4-9569-20394f20279d'],
 'embeddings': array([[ 0.00994719,  0.06914336, -0.05147117, ..., -0.03543337,
          0.01284807,  0.01248286],
        [ 0.00127744,  0.03129854, -0.02375378, ..., -0.00518363,
         -0.03280615,  0.02737713],
        [-0.10265923,  0.02650807,  0.02271505, ..., -0.03359748,
         -0.07984942, -0.01507713],
        [ 0.02123389, -0.02468547, -0.04494365, ..., -0.10995807,
          0.0057256 ,  0.09915376],
        [ 0.0187397 ,  0.0438284 , -0.04304256, ..., -0.07801615,
         -0.07840684, -0.00304187],
        [ 0.04981755,  0.11325381, -0.03532806, ..., -0.03498637,
          0.06544405,  0.05549667]], shape=(6, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history.

In [28]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Kolkata Knight Riders'}, page_content="Andre Russell is a powerful all-rounder known for his explosive batting and crucial bowling spells. Playing for Kolkata Knight Riders, he's won many matches single-handedly.")]

In [29]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.9693602323532104),
 (Document(metadata={'team': 'Kolkata Knight Riders'}, page_content="Andre Russell is a powerful all-rounder known for his explosive batting and crucial bowling spells. Playing for Kolkata Knight Riders, he's won many matches single-handedly."),
  1.0840948820114136)]

In [30]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  1.8436005115509033),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  1.890937328338623)]

In [32]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)


In [33]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['943b6c66-f7b9-4633-acc6-abba5a79cd69',
  'aa1921e6-a466-48e3-b6dc-f32ccfffb8de',
  '1239c316-d885-4baf-b588-f1df1c7b50d7',
  '4f3b4fd0-7036-46f1-b5e7-35327fbf69b7',
  '7e2401b2-7054-436d-b6e1-2f164217c5e1',
  '6f2fecdc-2444-45d4-9569-20394f20279d'],
 'embeddings': array([[ 0.00994719,  0.06914336, -0.05147117, ..., -0.03543337,
          0.01284807,  0.01248286],
        [ 0.00127744,  0.03129854, -0.02375378, ..., -0.00518363,
         -0.03280615,  0.02737713],
        [-0.10265923,  0.02650807,  0.02271505, ..., -0.03359748,
         -0.07984942, -0.01507713],
        [ 0.02123389, -0.02468547, -0.04494365, ..., -0.10995807,
          0.0057256 ,  0.09915376],
        [ 0.0187397 ,  0.0438284 , -0.04304256, ..., -0.07801615,
         -0.07840684, -0.00304187],
        [ 0.04981755,  0.11325381, -0.03532806, ..., -0.03498637,
          0.06544405,  0.05549667]], shape=(6, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history.

In [34]:
# delete document
vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

In [ ]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])